## Melon music data scraping from web
* 1. Scrap the title and id of 100 songs
* 2. Request 100 times(for each detail pages) >> get detail info
* 3. Store that to pandas data frame
* 4. and store that to DB as well (song table)

In [12]:
import requests, re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
# https://www.melon.com/album/detail.htm?albumId=10462799
# >>
# songId = 32790516
# >>
# https://www.melon.com/song/detail.htm?songId=32790516
url = 'https://www.melon.com/chart/index.htm'
req_header = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36',
}
html = requests.get(url, headers = req_header).text
# print(html)
soup = BeautifulSoup(html, 'html.parser')
soup.select("tbody tr td a")
# for body in soup.find_all('tbody'):
#     for tr in body.find_all("tr"):
#         for td in tr.find_all('td'):
#             for a in td.find_all('a'):
#                 print(a.string)
#                 # print(a_title)

# print(len(soup.select('#tb_list')))
# print(len(soup.select("#tb_list tr div.wrap_song_info a[href*='playSong']")))
links = []
for songs in soup.select("#tb_list tr div.wrap_song_info a[href*='playSong']"):
    links.append({
        'title': songs.string,
        'url' : "https://www.melon.com/song/detail.htm?songId=" + songs['href'][42:-2],
    })
    ## or use RE
    # href = songs['href']
    # matched = re.search(r"(\d+)\)",href)
    # print(matched.group(1))
# print(links)
song_lyric_lists = []
song_lyric_dict = {}
for link in links:
    html = requests.get(link['url'], headers= req_header).text
    soup = BeautifulSoup(html, 'html.parser')
    # print(soup)
    song_lyric_dict['곡명'] = link['title']
    song_lyric_dict['가수'] = soup.select('a[href*=".goArtistDetail"] span')[0].text
    song_lyric_dict['앨범'] = soup.select('div.meta dd')[0].text
    song_lyric_dict['발매일'] = soup.select('div.meta dd')[1].text
    song_lyric_dict['장르'] = soup.select('div.meta dd')[2].text
    #song_lyric_dict['좋아요'] = soup.select('span#d_like_count')[0].text
    lyric = soup.select('div#d_video_summary')[0].text
    regex = re.compile(r'[\n\r\t]')
    song_lyric_dict['가사'] = regex.sub('', lyric.strip())
    song_lyric_lists.append(song_lyric_dict)
    #print(song_lyric_dict)
    song_lyric_dict = {}
print(len(song_lyric_lists))


IndexError: list index out of range

In [10]:
# import re
# song_lyric_lists = []
# song_lyric_dict = {}
# for song in song_detail_lists:
#     html = requests.get(song['url'], headers=headers).text
#     soup = BeautifulSoup(html,'html.parser')
#     song_lyric_dict['곡명'] = song['title']
#     song_lyric_dict['가수'] = soup.select('a[href*=".goArtistDetail"] span')[0].text
#     song_lyric_dict['앨범'] = soup.select('div.meta dd')[0].text
#     song_lyric_dict['발매일'] = soup.select('div.meta dd')[1].text
#     song_lyric_dict['장르'] = soup.select('div.meta dd')[2].text
#     #song_lyric_dict['좋아요'] = soup.select('span#d_like_count')[0].text
#     lyric = soup.select('div#d_video_summary')[0].text
#     regex = re.compile(r'[\n\r\t]')
#     song_lyric_dict['가사'] = regex.sub('', lyric.strip())
#     song_lyric_lists.append(song_lyric_dict)
#     #print(song_lyric_dict)
#     song_lyric_dict = {}
# print(len(song_lyric_lists))